In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.cloud import bigquery, bigquery_storage
from pathlib import Path
from typing import List, Tuple, Dict
import logging

# Configuración del logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')


In [2]:
MES_A_EVALUAR = {'abril': 202104,
                 'mayo': 202105,
                 'junio': 202106}

MES_A_EVALUAR['abril']

202104

In [ ]:

# --- 1. CONFIGURACIÓN DEL PROYECTO ---
# Reemplaza estos valores con la configuración de tu config.yaml
PROJECT_ID = "dmecoyfin-250928192534125"
DATASET_ID = "dmeyf"
TABLE_TARGETS = "c03_features_historical" # Tabla que contiene foto_mes y clase_ternaria
OUTPUT_PATH = "gs://joaquinrk_data_bukito3/outputs" # Directorio donde guardas los CSV de PREDICT
MESES_A_EVALUAR = {'abril': 202104,
                 'mayo': 202105,
                 'junio': 202106}

MES_A_EVALUAR = 'abril'
# El nombre del archivo CSV generado por la fase PREDICT (ajusta si usas escenarios)
# Ejemplo para escenario simple:
PREDICT_CSV_FILE = f"{OUTPUT_PATH}/prediccion_{MES_A_EVALUAR}_{MESES_A_EVALUAR[MES_A_EVALUAR]}.csv"
# O si usas el caso simple: f"{OUTPUT_PATH}/{config.STUDY_NAME_OPTUNA}_CONSOLIDATED.csv"

# Parámetros de Negocio (de tu config.yaml)
GANANCIA_ACIERTO = 780000
COSTO_ESTIMULO = 20000


# --- 2. FUNCIÓN DE CARGA DE TARGETS DESDE BQ ---
def load_targets_from_bq(project: str, dataset: str, table: str, month: int) -> pd.DataFrame:
    """Carga numero_de_cliente y clase_ternaria para un mes específico."""
    logging.info(f"Cargando targets para el mes {month} desde BigQuery...")
    client = bigquery.Client(project=project)
    bqstorage_client = bigquery_storage.BigQueryReadClient()

    query = f"""
        SELECT
            numero_de_cliente,
            foto_mes,
            clase_ternaria
        FROM `{project}.{dataset}.{table}`
        WHERE foto_mes = {month}
    """

    job = client.query(query)
    arrow_table = job.result().to_arrow(bqstorage_client=bqstorage_client)
    df = arrow_table.to_pandas()

    # Filtrar registros nulos que puedan venir del target engineering
    df = df.dropna(subset=['clase_ternaria'])

    logging.info(f"Targets cargados: {len(df)} registros.")
    return df[['numero_de_cliente', 'clase_ternaria']]


# --- 3. FUNCIÓN DE CÁLCULO DE GANANCIA ---
def calculate_gain_curve(df_merged: pd.DataFrame) -> Tuple[np.ndarray, float]:
    """Calcula la ganancia acumulada y el máximo."""

    # 1. Asignar ganancia/costo por clase
    ganancia_individual = np.where(
        df_merged['clase_ternaria'] == "BAJA+2", GANANCIA_ACIERTO,
        np.where(df_merged['clase_ternaria'] != "BAJA+2", -COSTO_ESTIMULO, 0)
    )

    # 2. Ordenar por Probabilidad descendente
    df_merged['ganancia'] = ganancia_individual
    # Asume que 'y_pred_prob' es la columna con las probabilidades promedio
    df_sorted = df_merged.sort_values(by='y_pred_prob', ascending=False).reset_index(drop=True)

    # 3. Calcular Ganancia Acumulada
    ganancia_acumulada = df_sorted['ganancia'].cumsum()
    max_ganancia = ganancia_acumulada.max()

    return ganancia_acumulada.values, max_ganancia

# --- 4. FUNCIÓN DE PLOTEO ---
def plot_gain_curve(curve: np.ndarray, max_gain: float, month: int):
    """Genera la gráfica de la curva de ganancia."""
    plt.figure(figsize=(12, 6))

    # Identificar el punto de ganancia máxima
    k_max = np.argmax(curve) + 1

    plt.plot(curve, label=f'Ganancia Acumulada (Máx: {max_gain:,.0f} en k={k_max})')
    plt.axhline(0, color='red', linestyle='--', linewidth=0.8)
    plt.axvline(k_max, color='green', linestyle=':', linewidth=1.5, label=f'Punto Óptimo (k={k_max})')

    plt.title(f'Curva de Ganancia para el Mes: {month}')
    plt.xlabel('Cantidad de Clientes Estimulados (k)')
    plt.ylabel('Ganancia Acumulada')
    plt.legend()
    plt.grid(True, alpha=0.5)
    plt.show()


# --- 5. EJECUCIÓN PRINCIPAL ---
if __name__ == '__main__':

    # Paso 1: Cargar targets verdaderos desde BQ
    df_targets = load_targets_from_bq(PROJECT_ID, DATASET_ID, TABLE_TARGETS, MES_A_EVALUAR)

    # Paso 2: Cargar el archivo de predicciones (debe contener 'y_pred_prob' y 'numero_de_cliente')
    try:
        # ASUME QUE ESTE ARCHIVO CONTIENE LA COLUMNA 'y_pred_prob'
        # Si tu archivo se llama 'y_pred', ajústalo aquí.
        df_pred = pd.read_csv(PREDICT_CSV_FILE)

        # Renombrar columna si es necesario (ej. si PREDICT guardó la probabilidad como 'Predicted')
        if 'y_pred' in df_pred.columns:
             df_pred.rename(columns={'y_pred': 'y_pred_prob'}, inplace=True)
        elif 'Predicted' in df_pred.columns and df_pred['Predicted'].dtype != bool:
             df_pred.rename(columns={'Predicted': 'y_pred_prob'}, inplace=True)

        # Si solo tienes la predicción binaria (0/1), la curva no será suave. Se recomienda usar probabilidades.

        df_pred = df_pred[['numero_de_cliente', 'y_pred_prob']]
        logging.info(f"Predicciones cargadas: {len(df_pred)} registros.")

    except FileNotFoundError:
        logging.error(f"❌ Archivo de predicción no encontrado: {PREDICT_CSV_FILE}")
        exit()

    # Paso 3: Unir Targets y Predicciones
    df_merged = df_targets.merge(df_pred, on='numero_de_cliente', how='inner')
    logging.info(f"Registros unidos para evaluación: {len(df_merged)}")

    # Paso 4: Calcular Curva
    if not df_merged.empty:
        gain_curve, max_gain = calculate_gain_curve(df_merged)
        logging.info(f"🎉 Ganancia Máxima calculada para {MES_A_EVALUAR}: {max_gain:,.0f} pesos.")

        # Paso 5: Plotear
        plot_gain_curve(gain_curve, max_gain, MES_A_EVALUAR)
    else:
        logging.error("No hay registros coincidentes para generar la curva.")

In [3]:
df = pd.read_csv('gs://joaquinrk_data_bukito3/models/c03_consolidado_US_10porciento_hasta_febrero_2021/CONSOLIDATED/resumen_ganancias.csv')

In [4]:
df

,experimento,modelo,k_opt,ganancia_max,thr_opt,timestamp
0,c03_consolidado_US_10porciento_hasta_febrero_2...,lgb_top1_seed_151515,13005,260680000.0,0.863112,2025-11-23 18:22:39
1,c03_consolidado_US_10porciento_hasta_febrero_2...,lgb_top1_seed_155555,14295,265280000.0,0.833283,2025-11-23 18:22:40
2,c03_consolidado_US_10porciento_hasta_febrero_2...,lgb_top1_seed_515151,12308,285820000.0,0.858117,2025-11-23 18:22:41
3,c03_consolidado_US_10porciento_hasta_febrero_2...,lgb_top1_seed_551155,12285,287080000.0,0.859580,2025-11-23 18:22:42
4,c03_consolidado_US_10porciento_hasta_febrero_2...,lgb_top1_seed_555555,11372,267740000.0,0.874547,2025-11-23 18:22:43
5,c03_consolidado_US_10porciento_hasta_febrero_2...,lgb_top2_seed_151515,10445,260680000.0,0.893760,2025-11-23 18:22:44
6,c03_consolidado_US_10porciento_hasta_febrero_2...,lgb_top2_seed_155555,10215,278080000.0,0.908622,2025-11-23 18:22:45
7,c03_consolidado_US_10porciento_hasta_febrero_2...,lgb_top2_seed_515151,14221,272360000.0,0.830710,2025-11-23 18:22:46
8,c03_consolidado_US_10porciento_hasta_febrero_2...,lgb_top2_seed_551155,12267,267440000.0,0.867666,2025-11-23 18:22:48
9,c03_consolidado_US_10porciento_hasta_febrero_2...,lgb_top2_seed_555555,12757,260040000.0,0.863338,2025-11-23 18:22:49
